In [ ]:
from osgeo import gdal,osr
import os
import numpy as np
import glob
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

In [ ]:
import tensorflow.keras.backend as K
def r2(y_true, y_pred):
    a = K.square(y_pred - y_true)
    b = K.sum(a)
    c = K.mean(y_true)
    d = K.square(y_true - c)
    e = K.sum(d)
    f = 1 - b / e
    return f

def Write_Tiff_optimize(outname, data, X_size, Y_size, lon_min, lat_max, resolution, NoData_value=None):
    import os
    (filepath, tempfilename) = os.path.split(outname)
    if os.path.exists(filepath) == False:
        os.makedirs(filepath)
    ds = gdal.GetDriverByName('Gtiff').Create(outname, int(X_size), int(Y_size), 1, gdal.GDT_Float32)
    geotransfrom = (lon_min, resolution, 0, lat_max, 0, -resolution)
    ds.SetGeoTransform(geotransfrom)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    ds.SetProjection(srs.ExportToWkt())
    ds.GetRasterBand(1).WriteArray(data)
    if NoData_value != None:
        ds.GetRasterBand(1).SetNoDataValue(NoData_value)
    ds.FlushCache()
    ds = None

In [ ]:
model = tf.keras.models.load_model(r"*.h5", custom_objects={'r2':r2})

In [ ]:
train_data = pd.read_csv(r"*.csv")
X = train_data.iloc[:, 1:11]


In [ ]:
file_directory = r""
out_directory = r""
year = 2000
Albedo_list = np.array(sorted(glob.glob(file_directory + "//Albedo//" + str(year) + "\*.tiff")))
LAI_list = np.array(sorted(glob.glob(file_directory + "//LAI//" + str(year) + "\*.tif")))
r_list = np.array(sorted(glob.glob(file_directory + "//r//" + str(year) + "\*.tiff")))
sp_list = np.array(sorted(glob.glob(file_directory + "//sp//" + str(year) + "\*.tiff")))
# SSR_list = np.array(sorted(glob.glob(file_directory + "//SSR//" + str(year) + "\*.tif")))
strd_list = np.array(sorted(glob.glob(file_directory + "//strd//" + str(year) + "\*.tiff")))
ssrd_list = np.array(sorted(glob.glob(file_directory + "//ssrd//" + str(year) + "\*.tiff")))
t2m_list = np.array(sorted(glob.glob(file_directory + "//t2m//" + str(year) + "\*.tiff")))
ws_list = np.array(sorted(glob.glob(file_directory + "//ws//" + str(year) + "\*.tif")))
dem = r""
landcover = ""
landcover_data = gdal.Open(landcover).ReadAsArray().reshape(-1)
mete_region = r""
mete_region_data = gdal.Open(mete_region).ReadAsArray().reshape(-1)
china_region = gdal.Open(r"").ReadAsArray().reshape(-1)
file_directory = r""
LEc_list = np.array(sorted(glob.glob(file_directory + "//LE_C//" + "\*.tif")))
LEs_list = np.array(sorted(glob.glob(file_directory + "//LE_S//" + "\*.tif")))
date_list = np.array([Albedo_list[i].split("_")[-2].split(".")[0] for i in range(len(Albedo_list))])

In [ ]:
for file_i in range(0,366):
    Albedo_data = gdal.Open(Albedo_list[file_i]).ReadAsArray()
    shape = Albedo_data.shape
    Albedo_data = Albedo_data.reshape(-1)
    LAI_data = gdal.Open(LAI_list[file_i]).ReadAsArray().reshape(-1)
    r_data = gdal.Open(r_list[file_i]).ReadAsArray().reshape(-1)
    sp_data = gdal.Open(sp_list[file_i]).ReadAsArray().reshape(-1)
    # SSR_data = gdal.Open(SSR_list[file_i]).ReadAsArray().reshape(-1)
    ssrd_data = gdal.Open(ssrd_list[file_i]).ReadAsArray().reshape(-1)
    strd_data = gdal.Open(strd_list[file_i]).ReadAsArray().reshape(-1)
    t2m_data = gdal.Open(t2m_list[file_i]).ReadAsArray().reshape(-1)
    ws_data = gdal.Open(ws_list[file_i]).ReadAsArray().reshape(-1)
    dem_data = gdal.Open(dem).ReadAsArray().reshape(-1)
    landcover_data = gdal.Open(landcover).ReadAsArray().reshape(-1)
    
    LEc_data = gdal.Open(LEc_list[file_i]).ReadAsArray().reshape(-1)
    LEs_data = gdal.Open(LEs_list[file_i]).ReadAsArray().reshape(-1)
    LE_data = LEc_data + LEs_data
    print(LEs_list[file_i],LEc_list[file_i],date_list[file_i],"_begin")

    Mask = np.ones(Albedo_data.shape).astype(bool)
    Mask &= LE_data == 0
    Mask &= china_region == 1
    data = [Albedo_data[Mask],LAI_data[Mask],r_data[Mask],sp_data[Mask],ssrd_data[Mask],
            strd_data[Mask],t2m_data[Mask],ws_data[Mask],dem_data[Mask],landcover_data[Mask]]
    data=np.stack(data, 1)
    data = ((data - np.mean(X).values)/np.std(X).values)
    print(date_list[file_i],"_run")
    predict = model.predict(data)
    
    LEc_data[Mask] = predict[:,0]
    LEs_data[Mask] = predict[:,1]
        LEc_data = LEc_data.reshape(shape)
    LEs_data = LEs_data.reshape(shape)
    LE_data = LEc_data + LEs_data
    LE_data[LE_data <= 0] = 0
    LEc_data[LE_data <= 0] = 0
    LEs_data[LE_data <= 0] = 0
     
    outname = out_directory + '\\'+'LEc_DNN_'+date_list[file_i]+'.tif'
    Write_Tiff_optimize(outname, LEc_data, shape[1], shape[0], 73.0, 55.0, 0.01, 0)
    outname = out_directory + '\\'+'LEs_DNN_'+date_list[file_i]+'.tif'
    Write_Tiff_optimize(outname, LEs_data, shape[1], shape[0], 73.0, 55.0, 0.01, 0)
    outname = out_directory + '\\'+'LE_DNN_'+date_list[file_i]+'.tif'
    Write_Tiff_optimize(outname, LE_data, shape[1], shape[0], 73.0, 55.0, 0.01, 0)
    print(date_list[file_i],"_finish")